In [1]:
# Add parent directory
from sys import path as sysPath
from os import listdir
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic
from Machine_Score import main_machine_score as ms
from Machine_Score import direct_image_compare as dc
from Score_Analysis import main_score_analysis as sa
import main_SIMR as simr

# General modules
from os import path
import numpy as np
from matplotlib import pyplot as plt

sa.test()
gm.test()
im.test()
ic.test()
ms.test()
dc.test()
simr.test()

SA: Hi!  You're in Matthew's Main program for score analysis!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [17]:
tDirRaw = '../../1237678620102623480/'
tDir = gm.validPath(tDirRaw, printWarning=True)

print('Raw:  ',tDirRaw)
print('Full: ',tDir)

dataDir = '../../spam_data_1/'

Raw:   ../../1237678620102623480/
Full:  /mnt/c/Users/ogden/OneDrive/Documents/_Galaxy_Code/1237678620102623480/


In [3]:
tDir = gm.validPath('../targetDir/')
dataDir = '../../spam_data_1/'

tInfo = im.target_info_class( targetDir = tDir, printAll=True)

if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")

IM: target_info_class.__init__:
	 - targetDir:  /home/mbo2d/587722984435351614/
IM: Target.initTargetDir():
	 - targetDir: /home/mbo2d/587722984435351614/
	 - targetDir: (True) /home/mbo2d/587722984435351614/
	 - infoDir: (True) /home/mbo2d/587722984435351614/information/
	 - baseInfoLoc: (True) /home/mbo2d/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /home/mbo2d/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /home/mbo2d/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /home/mbo2d/587722984435351614/plots/
IM: Target: Opening target info json
Target Good!


In [4]:
# Going through all targets and gathering scores

tNames = []
tScores = []

tDirs = listdir( dataDir )

for tDir in tDirs:
    fullPath = gm.validPath( dataDir + tDir + '/' )
    
    tArg = gm.inArgClass()
    tArg.targetDir = fullPath
    
    tmpInfo = im.target_info_class( tArg = tArg )
    
    if tmpInfo.status:
        tNames.append( tmpInfo.get('target_id') )
        tScores.append( tmpInfo.getScores() )
    
    del tmpInfo

print("tNames: ", len(tNames), tNames)
print("tScores: ", len(tScores ))

for i in range( len( tNames ) ):
    print( tNames[i], tScores[i].columns)

FileNotFoundError: [Errno 2] No such file or directory: '../../spam_data_1/'

In [ ]:
tScores = tInfo.getScores()
print(tScores)

In [ ]:

#target_report_1(tInfo=tInfo)
tmp = '''
from os import listdir

dDir = gm.validPath( '../../tmp_scores' )
plotDir = gm.validPath( '../../tmp_plots' )
tList = listdir( dDir )
print(plotDir)

for tName in tList:
    
    sLoc = gm.validPath( dDir + tName + '/information/scores.csv' )
    pLoc = plotDir + tName + '.png'
    print(pLoc)
    
    if sLoc == None:
        continue
    
    #scores = gm.getScores(sLoc)
    
    target_report_1( scoreLoc = sLoc, plotLoc = pLoc )
'''



In [ ]:


def target_report_2( tInfo=None, printBase = True, printAll = False ):
    
    from copy import deepcopy
    
    scores = tInfo.getScores()
    scoreHeaders = list( scores.columns )
    allParams = tInfo.get('score_parameters')

    targKeys = []
    pertKeys = []
    unknKeys = []
    
    for pKey in allParams:
        if allParams[pKey]['scoreType'] == 'target':
            targKeys.append(pKey)
        elif allParams[pKey]['scoreType'] == 'perturbation':
            pertKeys.append(pKey)
        else:
            unknKeys.append(pKey)
    
    print("Target Machine Scores Found:", len(targKeys))
    for name in targKeys: gm.tabprint(name)
        
    print("Perturbation Scores Found:", len(pertKeys))
    for name in pertKeys: gm.tabprint(name)
        
    print("Unknown Scores Found:", len(unknKeys))
    for name in unknKeys: gm.tabprint(name)
    
    pertName = 'base_perturbation'
    if printBase: print("SA: target report 2: Hard coded perturbation: ",pertName)
        
    if allParams.get(pertName) == None:
        print("SA: target report 2: Base Perturbation Not found:")
        gm.tabprint('target: %s'%tInfo.get('target_id'))
        gm.tabprint("perturb: %s"%pertName)
        
    n = len(targKeys)
    
    fig, axs = plt.subplots( n+2, figsize=(9,7*(n+2)) )  
    
        
    # Show target image
    tImg = tInfo.getTargetImage( 'zoo' )
    axs[0].imshow(tImg, cmap='gray')
    axs[0].set_title("Target Image")

    # Show a model image
    rId = 'r00001'
    runArg = gm.inArgClass()
    runArg.setArg("printBase",False)
    rInfo = tInfo.getRunInfo(rID=rId, rArg = runArg)
    rImg = rInfo.getModelImage( 'zoo_0' )
    rN = scores.shape[0]

    if type(rImg) != type(None):
        axs[1].imshow(rImg, cmap='gray')
        axs[1].set_title('Model Image')

    # Go through and plot Target Machine Scores
    for i, sName in enumerate(targKeys):     
        
        hmScores = scores[['zoo_merger_score',sName,pertName]].dropna()
        hScores = hmScores['zoo_merger_score'].values
        mScores = hmScores[sName].values
        pScores = hmScores[pertName].values
        
        corr = np.corrcoef( hScores, mScores )[0,1]
        
        print('%s: %4d/%4d' % (sName,hmScores.shape[0],rN), corr)
        
        hs = { 'scores':hScores, 'name':'zoo_merger_score' }
        ms = { 'scores':mScores, 'name':sName }   
        ps = { 'scores':pScores, 'name':pertName }
                
        title = '%s: %4d/%4d ' % (sName,hmScores.shape[0],rN) 
        title += '\nCorr: %.4f' % corr
        
        sa.basicHeatSubPlot( fig, axs[i+2], hs, ms, ps, titleName=title)        
        
    return None  

    plt.tight_layout()
    
    if tInfo != None:
        plotLoc = tInfo.plotDir + 'basic_target_report_2.pdf'
        print('PLotLoc: ', plotLoc)
        fig.savefig(plotLoc, bbox_inches='tight')
        
target_report_2( tInfo = tInfo )